In [1]:
import pandas as pd
from pathlib import Path
import os

In [2]:
os.getcwd()

'C:\\Users\\colby\\Downloads\\pandas wk 4\\Starter_Code\\PyCitySchools'

In [3]:
school_data_to_load = "Resources/schools_complete.csv"
school_data = pd.read_csv(school_data_to_load)

In [5]:
student_data_to_load = "Resources/students_complete.csv"
student_data = pd.read_csv(student_data_to_load)


In [6]:
school_data

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [7]:
student_data

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90


In [18]:
# merge data
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [65]:
# Calculate District Summary
total_schools = len(school_data_complete.school_name.unique())
total_students = school_data_complete.student_name.count()
total_budget = sum(school_data_complete.budget.unique())
average_math_score = school_data_complete.math_score.mean()
average_reading_score = school_data_complete.reading_score.mean()
passing_math_pct = (school_data_complete[school_data_complete['math_score'] >= 70].math_score.count()) / total_students
passing_reading_pct = (school_data_complete[school_data_complete['reading_score'] >= 70].reading_score.count()) / total_students
passing_math_reading_count = school_data_complete[(school_data_complete["math_score"]>=70) & (school_data_complete["reading_score"]>=70)].count()["student_name"]
overall_passing_rate = passing_math_reading_count / float(total_students)*100




In [44]:
district_summary = pd.DataFrame({"Total Schools":[total_schools],
                                 "Total Students":[total_students],
                                 "Total Budget":[total_budget],
                                 "Average Math Score":[average_math_score],
                                 "Average Reading Score":[average_reading_score],
                                 "% Passing Math":[passing_math_pct*100],
                                 "% Passing Reading":[passing_reading_pct*100],
                                 "% Overall Passing Rate":[overall_passing_rate*100]})

In [45]:
district_summary['Total Students'] = district_summary.apply(lambda x: "{:,.0f}".format(x['Total Students']), axis=1)
district_summary['Total Budget'] = district_summary.apply(lambda x: "${:,.2f}".format(x['Total Budget']), axis=1)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


In [33]:
#calculaate School Summary
school_summary = school_data_complete.groupby(['school_name'])
school_names = school_data_complete.school_name.sort_values().unique()
school_types = school_data.sort_values(by="school_name").type
school_total_students = list(school_summary.student_name.count())
school_budget = list(school_summary.budget.mean())
school_per_student_budget = [i/j for i,j in zip(school_budget,school_total_students)]
school_avg_math_score = list(school_summary.math_score.mean())
school_avg_reading_score = list(school_summary.reading_score.mean())

In [35]:
# Calculating passing percentages
school_summary = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name'])
school_pct_passing_math = [(i/j)*100 for i,j in zip(school_summary.math_score.count(),school_total_students)]
school_summary = school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name'])
school_pct_passing_reading = [(i/j)*100 for i,j in zip(school_summary.reading_score.count(),school_total_students)]
school_overall_passing = [(i+j)/2 for i,j in zip(school_pct_passing_math,school_pct_passing_reading)]

In [36]:
school_summary_df = pd.DataFrame({"School Names":school_names,
                                  "School Type":school_types,
                                  "Total Students":school_total_students,
                                  "Total School Budget":school_budget,
                                  "Per Student Budget":school_per_student_budget,
                                  "Average Math Score":school_avg_math_score,
                                  "Average Reading Score":school_avg_reading_score,
                                  "% Passing Math":school_pct_passing_math,
                                  "% Passing Reading":school_pct_passing_reading,
                                  "Overall Passing Rate":school_overall_passing})

school_summary_df = school_summary_df.reset_index(drop=True)
school_summary_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


In [85]:
school_types = school_data.set_index(["school_name"])["type"]
per_school_counts = list(school_summary.student_name.count())
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts
per_school_math =  school_data_complete.math_score.mean()
per_school_reading = school_data_complete.reading_score.mean()
student_passing_math = len(school_data_complete[school_data_complete["math_score"] >= 70])
student_passing_reading =  len(school_data_complete[school_data_complete["reading_score"] >= 70])
passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]

In [37]:
top_5 = school_summary_df.sort_values(by='Overall Passing Rate', ascending=False).head(5).reset_index(drop=True)
top_5

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
1,Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
2,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
3,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
4,Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [38]:
bot_5 = school_summary_df.sort_values(by='Overall Passing Rate', ascending=True).head(5).reset_index(drop=True)
bot_5

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
3,Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
4,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [50]:
def average_math_by_grade(grade):
    school_summary = school_data_complete.loc[school_data_complete.grade == grade].groupby(['school_name'])
    school_names = school_data_complete.school_name.sort_values().unique()
    school_avg_math_score = list(school_summary.math_score.mean())
    average_math_df = pd.DataFrame({"School Names":school_names,
        f"{grade} Avg Math Score":school_avg_math_score})
    average_math_df = average_math_df.reset_index(drop=True)
    return average_math_df

In [52]:
def average_reading_by_grade(grade):
    school_summary = school_data_complete.loc[school_data_complete.grade == grade].groupby(['school_name'])
    school_names = school_data_complete.school_name.sort_values().unique()
    school_avg_reading_score = list(school_summary.reading_score.mean())
    average_reading_df = pd.DataFrame({"School Names":school_names,
                                    f"{grade} Avg Reading Score":school_avg_reading_score})
    average_reading_df = average_reading_df.reset_index(drop=True)
    return average_reading_df

In [66]:
grade_9 = average_math_by_grade('9th')
grade_10 = average_math_by_grade('10th')
grade_11 = average_math_by_grade('11th')
grade_12 = average_math_by_grade('12th')
avg_math_score_by_grade = pd.merge(grade_9,grade_10,how='inner',suffixes=('',''))
avg_math_score_by_grade = pd.merge(avg_math_score_by_grade,grade_11,how='inner',suffixes=('',''))
avg_math_score_by_grade = pd.merge(avg_math_score_by_grade,grade_12,how='inner',suffixes=('',''))
avg_math_score_by_grade

,School Names,9th Avg Math Score,10th Avg Math Score,11th Avg Math Score,12th Avg Math Score
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [67]:
grade_9 = average_reading_by_grade('9th')
grade_10 = average_reading_by_grade('10th')
grade_11 = average_reading_by_grade('11th')
grade_12 = average_reading_by_grade('12th')
avg_reading_score_by_grade = pd.merge(grade_9,grade_10,how='inner',suffixes=('',''))
avg_reading_score_by_grade = pd.merge(avg_reading_score_by_grade,grade_11,how='inner',suffixes=('',''))
avg_reading_score_by_grade = pd.merge(avg_reading_score_by_grade,grade_12,how='inner',suffixes=('',''))
avg_reading_score_by_grade

,School Names,9th Avg Reading Score,10th Avg Reading Score,11th Avg Reading Score,12th Avg Reading Score
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [130]:
# SCORES BY SCHOOL SIZE
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [131]:
school_spending_df = school_summary_df.copy()
school_spending_df = school_spending_df[["School Names", "School Type", "Total Students", "Total School Budget", "Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading",
                                     "Overall Passing Rate"]]

In [134]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_per_student_budget, spending_bins, labels)
school_spending_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Ranges (Per Student)
0,Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,"(585, 630]"
1,Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,"(0, 585]"
2,Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,"(630, 645]"
3,Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,"(630, 645]"
4,Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,"(585, 630]"
5,Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,"(645, 680]"
6,Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,"(0, 585]"
7,Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,"(645, 680]"
8,Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,"(645, 680]"
9,Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270,"(585, 630]"


In [137]:
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Overall Passing Rate"]

In [142]:
spending_summary_df = pd.DataFrame({"Average Math Score":spending_math_scores,
                                  "Average Reading Score":spending_reading_scores,
                                  "% Passing Math":spending_passing_math,
                                  "% Passing Reading":spending_passing_reading,
                                  "Overall Passing Rate":overall_passing_spending})
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
"(0, 585]",83.455399,83.933814,93.460096,96.610877,95.035486
"(585, 630]",81.899826,83.155286,87.133538,92.718205,89.925871
"(630, 645]",78.518855,81.624473,73.484209,84.391793,78.938001
"(645, 680]",76.997210,81.027843,66.164813,81.133951,73.649382


In [174]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], size_bins, labels=labels)
grouped_size_df = school_summary_df.groupby(["School Size"])   
size_summary_df = grouped_size_df.mean()
size_summary_df[["Average Math Score",
                "Average Reading Score",
                "% Passing Math",
                "% Passing Reading",
                "Overall Passing Rate"]]


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [109]:
scores_by_type = school_summary_df[["School Names",
                                    "School Type",
                                    "Average Math Score",
                                    "Average Reading Score",
                                    "% Passing Math",
                                    "% Passing Reading",
                                    "Overall Passing Rate"]]
scores_by_type = scores_by_type.groupby("School Type")
scores_by_type.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
